
Male and female should have distinct confusion matrix and accuracies.
Proposed approach

Done!!

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Sep  7 15:39:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!7z e "/content/drive/My Drive/celeba_png/img_align_celeba_png.7z.001" -o"/content/img/" -mmt=10

In [ ]:
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mt
from skimage import io, transform


import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


# additional files is located in /home/jamal/Downloads/additional_files/
def save_checkpoint_high(state, filename='/content/drive/MyDrive/vre_autoencoder_final/trained_modes_final/checkpoint_propose_Male_celeb_high.pth.tar'):
    torch.save(state, filename)
def save_checkpoint_low(state, filename='/content/drive/MyDrive/vre_autoencoder_final/trained_modes_final/checkpoint_propose_Male_celeb_low.pth.tar'):
    torch.save(state, filename)
def save_checkpoint_class_MF(state, filename='/content/drive/MyDrive/vre_autoencoder_final/trained_modes_final/checkpoint_classification_propose_MF.pth.tar'):
    torch.save(state, filename)
def save_checkpoint_class_SD(state, filename='/content/drive/MyDrive/vre_autoencoder_final/trained_modes_final/checkpoint_classification_propose_SD.pth.tar'):
    torch.save(state, filename)
def save_checkpoint_class_YO(state, filename='/content/drive/MyDrive/vre_autoencoder_final/trained_modes_final/checkpoint_classification_propose_YO.pth.tar'):
    torch.save(state, filename)

file_path ='/content/drive/MyDrive/list_attr_celeba1_train_used.txt'


images_path = '/content/img/' #train

load_net = 0

columns = ['ImgId','5_o_Clock_Shadow',
           ' Arched_Eyebrows', 'Attractive',
           'Bags_Under_Eyes', 'Bald', 'Bangs',
           'Big_Lips', 'Big_Nose', 'Black_Hair',
           'Blond_Hair', 'Blurry', 'Brown_Hair',
           'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
           'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup',
           'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache',
           'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose',
           'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair',
           'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']


cele_attrib = pd.read_csv(file_path,delimiter = "\s+",names = columns)# lfw = cele_attrib.set_index('SUBJECT_ID')
lfw = cele_attrib


import torch
import torch.nn.functional as F
import torch.nn as nn

class VAE(nn.Module):
    def __init__(self, nc, ngf, ndf, latent_variable_size):
        super(VAE, self).__init__()

        self.nc = nc
        self.ngf = ngf
        self.ndf = ndf
        self.latent_variable_size = latent_variable_size

        # encoder
        self.e1 = nn.Conv2d(nc, self.ndf, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(self.ndf)

        self.e2 = nn.Conv2d(self.ndf, self.ndf*2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(self.ndf*2)

        self.e3 = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf*4)

        self.e4 = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf*8)

        self.e5 = nn.Conv2d(ndf*8, ndf*8, 4, 2, 1)
        self.bn5 = nn.BatchNorm2d(ndf*8)

        self.fc1 = nn.Linear(ndf*8*4*4, latent_variable_size)
        self.fc2 = nn.Linear(ndf*8*4*4, latent_variable_size)

        # decoder
        self.d1 = nn.Linear(latent_variable_size, self.ngf*8*2*4*4)

        self.up1 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd1 = nn.ReplicationPad2d(1)
        self.d2 = nn.Conv2d(self.ngf*8*2, self.ngf*8, 3, 1)
        self.bn6 = nn.BatchNorm2d(self.ngf*8, 1.e-3)

        self.up2 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd2 = nn.ReplicationPad2d(1)
        self.d3 = nn.Conv2d(self.ngf*8, self.ngf*4, 3, 1)
        self.bn7 = nn.BatchNorm2d(self.ngf*4, 1.e-3)

        self.up3 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd3 = nn.ReplicationPad2d(1)
        self.d4 = nn.Conv2d(self.ngf*4, self.ngf*2, 3, 1)
        self.bn8 = nn.BatchNorm2d(self.ngf*2, 1.e-3)

        self.up4 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd4 = nn.ReplicationPad2d(1)
        self.d5 = nn.Conv2d(self.ngf*2, self.ngf, 3, 1)
        self.bn9 = nn.BatchNorm2d(self.ngf, 1.e-3)

        self.up5 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd5 = nn.ReplicationPad2d(1)
        self.d6 = nn.Conv2d(self.ngf, nc, 3, 1)

        self.leakyrelu = nn.LeakyReLU(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.leakyrelu(self.bn1(self.e1(x)))
        h2 = self.leakyrelu(self.bn2(self.e2(h1)))
        h3 = self.leakyrelu(self.bn3(self.e3(h2)))
        h4 = self.leakyrelu(self.bn4(self.e4(h3)))
        h5 = self.leakyrelu(self.bn5(self.e5(h4)))
        h5 = h5.view(-1, self.ndf*8*4*4)

        return self.fc1(h5), self.fc2(h5), h5

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if use_cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h1 = self.relu(self.d1(z))
        h1 = h1.view(-1, self.ngf*8*2, 4, 4)
        h2 = self.leakyrelu(self.bn6(self.d2(self.pd1(self.up1(h1)))))
        h3 = self.leakyrelu(self.bn7(self.d3(self.pd2(self.up2(h2)))))
        h4 = self.leakyrelu(self.bn8(self.d4(self.pd3(self.up3(h3)))))
        h5 = self.leakyrelu(self.bn9(self.d5(self.pd4(self.up4(h4)))))

        return self.sigmoid(self.d6(self.pd5(self.up5(h5))))

    def get_latent_var(self, x):
        mu, logvar, h1 = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        return z

    def forward(self, x):
        mu, logvar,h2 = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        res = self.decode(z)
        return res, mu, logvar, h2


class VAE_low(nn.Module):
    def __init__(self, nc, ngf, ndf, latent_variable_size):
        super(VAE_low, self).__init__()

        self.nc = nc
        self.ngf = ngf
        self.ndf = ndf
        self.latent_variable_size = latent_variable_size

        # encoder
        self.e1 = nn.Conv2d(nc, self.ndf, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(self.ndf)

        self.e2 = nn.Conv2d(self.ndf, self.ndf*2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(self.ndf*2)

        self.e3 = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf*4)
        #
        # self.e4 = nn.Conv2d(ndf*4, ndf*4, 4, 2, 1)
        # self.bn4 = nn.BatchNorm2d(ndf*4)

        # self.e5 = nn.Conv2d(ndf*8, ndf*8, 4, 2, 1)
        # self.bn5 = nn.BatchNorm2d(ndf*8)

        self.fc1 = nn.Linear(ndf*4*4, latent_variable_size)
        self.fc2 = nn.Linear(ndf*4*4, latent_variable_size)

        # decoder
        self.d1 = nn.Linear(latent_variable_size, ngf*4*4)

        self.up1 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd1 = nn.ReplicationPad2d(1)
        self.d2 = nn.Conv2d(self.ngf, self.ngf//2, 3, 1)
        self.bn6 = nn.BatchNorm2d(self.ngf//2, 1.e-3)

        self.up2 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd2 = nn.ReplicationPad2d(1)
        self.d3 = nn.Conv2d(self.ngf, self.ngf//2, 3, 1)
        self.bn7 = nn.BatchNorm2d(self.ngf//2, 1.e-3)

        self.up3 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd3 = nn.ReplicationPad2d(1)
        self.d4 = nn.Conv2d(self.ngf*4, self.ngf*2, 3, 1)
        self.bn8 = nn.BatchNorm2d(self.ngf*2, 1.e-3)

        self.up4 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd4 = nn.ReplicationPad2d(1)
        self.d5 = nn.Conv2d(self.ngf*2, self.ngf, 3, 1)
        self.bn9 = nn.BatchNorm2d(self.ngf, 1.e-3)

        self.up5 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd5 = nn.ReplicationPad2d(1)
        self.d6 = nn.Conv2d(self.ngf//2, nc, 3, 1)

        self.leakyrelu = nn.LeakyReLU(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.leakyrelu(self.bn1(self.e1(x)))
        h2 = self.leakyrelu(self.bn2(self.e2(h1)))
        h3 = self.leakyrelu(self.bn3(self.e3(h2)))
        # h4 = self.leakyrelu(self.bn4(self.e4(h3)))
        # h5 = self.leakyrelu(self.bn5(self.e5(h4)))
        h5 = h3.view(-1, self.ndf*4*4)

        return self.fc1(h5), self.fc2(h5),h5

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if use_cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h1 = self.relu(self.d1(z))
        h1 = h1.view(-1, self.ngf, 4, 4)
        h2 = self.leakyrelu(self.bn6(self.d2(self.pd1(self.up1(h1)))))


        return self.sigmoid(self.d6(self.pd5(self.up5(h2))))

    def get_latent_var(self, x):
        mu, logvar,h5 = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        return z

    def forward(self, x):
        mu, logvar, h5 = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        res = self.decode(z)
        return res, mu, logvar, h5

class Net_A(nn.Module):

    def __init__(self):
        super(Net_A,self).__init__()

        self.ip3 = nn.Linear(2*560,2000,True)   # change the first parameter in case you change the size of the small image
        self.n3 = nn.BatchNorm1d(2000)
        self.n1 = nn.BatchNorm1d(2*560)
        self.ip4 = nn.Linear(2000,1000,True)
        self.n4 = nn.BatchNorm1d(1000)
        self.ip5 = nn.Linear(1000,128,True)
        self.n5 = nn.BatchNorm1d(128)
        self.ip6 = nn.Linear(128,2,True)

    def forward(self,x):
        x = self.n1(x)
        x2 = self.ip3(x)
        x2 = F.relu(x2)
        x2 = self.n3(x2)

        x2 = self.ip4(x2)
        x2 = F.relu(x2)
        x2 = self.n4(x2)

        x2 = self.ip5(x2)
        x2 = F.relu(x2)
        x2 = self.n5(x2)

        x2 = self.ip6(x2)
        x2 = F.relu(x2)




        # x2 = x2.mul(-1)
        # x = F.relu(x)
        # x = F.softmax(x,1)
        return x2


class Male_Female_dataset(Dataset):


   def __init__(self,root_dir,shape,transform=None):
     self.root_dir = root_dir
     self.transform = transform
     self.shape = shape

   def __len__(self):
     return len(cele_attrib)

   def __getitem__(self,idx):
    img1_name = os.path.join(self.root_dir,("{:06d}.png".format(idx+1)))
    image1 = io.imread(img1_name)
    image1 = transform.resize(image1,ip_shape)
    #print("Got image 1: {0}".format(image1))


    t = torch.rand(1)
    if t > 0.5:
        h = torch.randint(0,len(self),(1,1))
        img2_name = os.path.join(self.root_dir,("{:06d}.png".format(int(h)+1)))
        image2 = io.imread(img2_name)
        #print("Got image 2: {0}".format(image2))
        image2 = transform.resize(image2,self.shape)
        image2 = torch.Tensor.float(torch.from_numpy(image2))
        image2 = (torch.Tensor.permute(image2,(2,0,1)))
        image2 = image2/256.0
        image1 = torch.Tensor.float(torch.from_numpy(image1))
        image1 = (torch.Tensor.permute(image1,(2,0,1)))
        image1 = image1/256.0
        annot = 0
        gender = cele_attrib['Male'][idx]
        young = cele_attrib['Young'][idx]

    else:
        image2 = image1
        image2 = transform.resize(image2,self.shape)
        image2 = torch.Tensor.float(torch.from_numpy(image2))
        image2 = (torch.Tensor.permute(image2,(2,0,1)))
        image2 = image2/256.0
        image1 = torch.Tensor.float(torch.from_numpy(image1))
        image1 = (torch.Tensor.permute(image1,(2,0,1)))
        image1 = image1/256.0
        annot = 1
        gender = cele_attrib['Male'][idx]
        young = cele_attrib['Young'][idx]

    sample = {'image1' : image1,
                'image2' : image2,
                'same'   : annot,
                'gender' : gender,
                'young' : young}
    return sample




reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False


def loss_function(recon_x, x, mu, logvar,loss1,loss2,loss_yo):
    BCE = reconstruction_function(recon_x, x)

    # https://arxiv.org/abs/1312.6114 (Appendix B)
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD +loss1- 0.1* loss2 -0.0002*loss_yo


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
shape = (16,16)
ip_shape = (128,128)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 1-train_ratio-val_ratio
dataset = Male_Female_dataset(images_path,shape)


if load_net:
    model_high = VAE(nc=3, ngf=ip_shape[0], ndf=ip_shape[1], latent_variable_size=500).to(device)
    model_low = VAE_low(nc=3, ngf=shape[0], ndf=shape[1], latent_variable_size=60).to(device)
    class_net_SD = Net_A().to(device)
    class_net_MF = Net_A().to(device)
    class_net_YO = Net_A().to(device)

    checkpoint_high = torch.load('/content/drive/MyDrive/vre_autoencoder_final/trained_modes/checkpoint_propose_Male_celeb_high.pth.tar')
    checkpoint_low = torch.load('/content/drive/MyDrive/vre_autoencoder_final/trained_modes/checkpoint_propose_Male_celeb_low.pth.tar')
    checkpoint_class_SD = torch.load('/content/drive/MyDrive/vre_autoencoder_final/trained_modes/checkpoint_classification_propose_SD.pth.tar')
    checkpoint_class_SD = torch.load('/content/drive/MyDrive/vre_autoencoder_final/trained_modes/checkpoint_classification_propose_MF.pth.tar')
    checkpoint_class_YO = torch.load('/content/drive/MyDrive/vre_autoencoder_final/trained_modes/checkpoint_classification_propose_YO.pth.tar')

    model_high.load_state_dict ( checkpoint_high['state_dict'])
    model_low.load_state_dict ( checkpoint_low['state_dict'])
    class_net_SD.load_state_dict ( checkpoint_class_SD['state_dict'])
    class_net_MF.load_state_dict ( checkpoint_class_SD['state_dict'])
    class_net_YO.load_state_dict ( checkpoint_class_SD['state_dict'])

    optimizer_high = optim.Adam(model_high.parameters(),lr = 0.00001, weight_decay = 0.0005)
    optimizer_low = optim.Adam(model_low.parameters(),lr = 0.0001, weight_decay = 0.0005)
    optimizer_class_SD = optim.Adam(class_net_SD.parameters(),lr = 0.0001, weight_decay = 0.0005)
    optimizer_class_MF = optim.Adam(class_net_SD.parameters(),lr = 0.0001, weight_decay = 0.0005)
    optimizer_class_YO = optim.Adam(class_net_SD.parameters(),lr = 0.0001, weight_decay = 0.0005)

    optimizer_high.load_state_dict = checkpoint_high['optimizer']
    optimizer_low.load_state_dict = checkpoint_low['optimizer']
    optimizer_class_SD.load_state_dict = checkpoint_class_SD['optimizer']
    optimizer_class_MF.load_state_dict = checkpoint_class_SD['optimizer']
    optimizer_class_YO.load_state_dict = checkpoint_class_SD['optimizer']



else:
    #torch.cuda.empty_cache()
    model_high   = VAE(nc=3, ngf=ip_shape[0], ndf=ip_shape[1], latent_variable_size=500).to(device)
    model_low    = VAE_low(nc=3, ngf=shape[0], ndf=shape[1], latent_variable_size=60).to(device)
    class_net_SD = Net_A().to(device)
    class_net_MF = Net_A().to(device)
    class_net_YO = Net_A().to(device)


    optimizer_high     = optim.Adam(model_high.parameters(),lr = 0.00001, weight_decay = 0.0005)
    optimizer_low      = optim.Adam(model_low.parameters(),lr = 0.00001, weight_decay = 0.0005)
    optimizer_class_SD = optim.Adam(class_net_SD.parameters(),lr = 0.000001, weight_decay = 0.0005)
    optimizer_class_MF = optim.Adam(class_net_MF.parameters(),lr = 0.000001, weight_decay = 0.0005)
    optimizer_class_YO = optim.Adam(class_net_YO.parameters(),lr = 0.000001, weight_decay = 0.0005)

    checkpoint_class_SD= {'epoch':0}



index             = np.random.permutation(len(dataset)-5)
train_data_length = int(train_ratio*len(index))
val_data_length   = int(val_ratio*len(index))
test_data_length  = int(test_ratio*len(index))
train_index       = index[:train_data_length]
val_index         = index[train_data_length:(train_data_length+val_data_length)]
test_index        = index[train_data_length+val_data_length:]

train_dataloader = DataLoader(dataset,batch_size=50,sampler = SubsetRandomSampler(train_index))

err_same = []
err_gender = []

acc1_SD = []
acc1_MF = []
acc1_YO = []


criterion2 = nn.CrossEntropyLoss()
t = -1
# Check if images are already exist in the folder if not then pass
for ep in range(checkpoint_class_SD['epoch'],5):
    for i,data in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        optimizer_high.zero_grad()
        optimizer_low.zero_grad()
        optimizer_class_SD.zero_grad()
        optimizer_class_MF.zero_grad()

        input1, input2 , label, gender ,young= data.items()
        input1, input2 = input1[1].to(device), input2[1].to(device)
        label1 = label[1].to(device)
        gender = gender[1].to(device)
        young = young[1].to(device)
        gender = (gender+torch.ones_like(gender).to(device))//2
        young = (young+torch.ones_like(young).to(device))//2



        recon_batch_high, mu_high, logvar_high, h2_h = model_high(input1)
        recon_batch_low, mu_low, logvar_low, h2_l = model_low(input2)

        mu_all = torch.cat((mu_high,mu_low),1)
        logvar_all = torch.cat((logvar_high,logvar_low),1)
        class_input = torch.cat((mu_all,logvar_all),1)
        #class_input = mu_all
        class_out_SD = class_net_SD(class_input)
        class_out_MF = class_net_MF(class_input)
        class_out_YO = class_net_YO(class_input)



        loss_class_SD = criterion2(class_out_SD,label1)
        loss_class_MF = criterion2(class_out_MF,gender)
        loss_class_YO = criterion2(class_out_YO,young)

        loss_high = loss_function(recon_batch_high, input1, mu_high, logvar_high,loss_class_SD,loss_class_MF,loss_class_YO)
        loss_low = loss_function(recon_batch_low, input2, mu_low, logvar_low,loss_class_SD,loss_class_MF,loss_class_YO)

        loss_class_SD.backward(retain_graph=True)
        loss_class_MF.backward(retain_graph=True)
        loss_high.backward(retain_graph=True)
        loss_low.backward()


        optimizer_high.step()
        optimizer_low.step()
        optimizer_class_SD.step()
        optimizer_class_MF.step()
        optimizer_class_YO.step()



        if (i%5==0):
            print ("HQ image reconstruction loss:",loss_high.item(),'iteration: ',i,"epoch: ",ep)
            print ("LQ image reconstruction loss:",loss_low.item(),'iteration: ',i,"epoch: ",ep)
            print ("Same Different Classification loss:", loss_class_SD.item(),'iteration: ',i,"epoch: ",ep)
            print ("Male Female Classification loss:", loss_class_MF.item(),'iteration: ',i,"epoch: ",ep)
            print ("Young Classification loss:", loss_class_YO.item(),'iteration: ',i,"epoch: ",ep)
            print ("----------------------------------------------------")

        if (i%50==0):
            val_index1 = np.random.permutation(val_index)[:100]
            val_dataloader = DataLoader(dataset,batch_size=100,sampler = SubsetRandomSampler(val_index1))
            val_iter = iter(val_dataloader)

            total = 0
            correct1_SD = 0
            correct1_MF = 0
            correct1_YO = 0
            for j,dataj in enumerate(val_dataloader):
                input1j, input2j, labelj, gender, young = dataj.items()
                input1j, input2j = input1j[1].to(device), input2j[1].to(device)

                labelj = labelj[1].to(device)
                gender = gender[1].to(device)
                gender = (gender+torch.ones_like(gender).to(device))//2

                young = young[1].to(device)
                young = (young+torch.ones_like(young).to(device))//2
                recon_batch_high, mu_high, logvar_high, h2_h = model_high(input1j)
                recon_batch_low, mu_low, logvar_low, h2_l = model_low(input2j)

                mu_all = torch.cat((mu_high,mu_low.data),1)
                logvar_all = torch.cat((logvar_high,logvar_low),1)
                class_input = torch.cat((mu_all,logvar_all),1)
                #class_input = mu_all
                class_out_SD = class_net_SD(class_input)
                class_out_MF = class_net_MF(class_input)
                class_out_YO = class_net_YO(class_input)
                _,predicted1_SD = torch.max(class_out_SD.data,1)
                _,predicted1_MF = torch.max(class_out_MF.data,1)
                _,predicted1_YO = torch.max(class_out_YO.data,1)
                # _,predicted2 = torch.max(output_A.data,1)
                total +=labelj.size(0)
                correct1_SD += (predicted1_SD == labelj).sum().item()
                correct1_MF += (predicted1_MF == gender).sum().item()
                correct1_YO += (predicted1_YO == young).sum().item()
                # correct2 += (predicted2 == gender).sum().item()
            print('Accuracy of SD: %d %%'%(100*correct1_SD/total))
            print('Accuracy of MF: %d %%'%(100*correct1_MF/total))
            print('Accuracy of YO: %d %%'%(100*correct1_MF/total))

            # print('Accuracy_MF: %d %%'%(100*correct2/total))

            acc1_SD.append(100*correct1_SD/total) #low high
            acc1_MF.append(100*correct1_MF/total) #low high
            acc1_YO.append(100*correct1_YO/total) #low high
            print('Mean Accuracy of SD: %d %%'%(np.mean(acc1_SD)))
            print('Mean Accuracy of MF: %d %%'%(np.mean(acc1_MF)))
            print('Mean Accuracy of YO: %d %%'%(np.mean(acc1_YO)))
            print("----------------------------------------------------")
            # acc2.append(100*correct2/total) # male female
    save_checkpoint_class_SD({
            'epoch': ep + 1,
            'state_dict': class_net_SD.state_dict(),

            'optimizer' : optimizer_class_SD.state_dict(),
        })
    save_checkpoint_class_MF({
            'epoch': ep + 1,
            'state_dict': class_net_MF.state_dict(),

            'optimizer' : optimizer_class_MF.state_dict(),
        })
    save_checkpoint_class_YO({
            'epoch': ep + 1,
            'state_dict': class_net_YO.state_dict(),

            'optimizer' : optimizer_class_YO.state_dict(),
        })
    save_checkpoint_high({
            'epoch': ep + 1,
            'state_dict': model_high.state_dict(),

            'optimizer' : optimizer_high.state_dict(),
        })
    save_checkpoint_low({
            'epoch': ep + 1,
            'state_dict': model_low.state_dict(),

            'optimizer' : optimizer_low.state_dict(),
        })

dataset.__getitem__(1)

cele_attrib['Male'][1]

cuda


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:461: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:462: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


HQ image reconstruction loss: 3235.7119140625 iteration:  0 epoch:  0
LQ image reconstruction loss: 408.0064392089844 iteration:  0 epoch:  0
Same Different Classification loss: 0.7301753759384155 iteration:  0 epoch:  0
Male Female Classification loss: 0.7471131086349487 iteration:  0 epoch:  0
Young Classification loss: 0.7548452019691467 iteration:  0 epoch:  0
----------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:523: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:526: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


Streaming output truncated to the last 5000 lines.
LQ image reconstruction loss: 0.8042982816696167 iteration:  1950 epoch:  3
Same Different Classification loss: 0.2930217385292053 iteration:  1950 epoch:  3
Male Female Classification loss: 0.2758033871650696 iteration:  1950 epoch:  3
Young Classification loss: 3.3422582149505615 iteration:  1950 epoch:  3
----------------------------------------------------
Accuracy of SD: 91 %
Accuracy of MF: 96 %
Accuracy of YO: 96 %
Mean Accuracy of SD: 64 %
Mean Accuracy of MF: 80 %
Mean Accuracy of YO: 31 %
----------------------------------------------------
HQ image reconstruction loss: 0.41465452313423157 iteration:  1955 epoch:  3
LQ image reconstruction loss: 0.7442613244056702 iteration:  1955 epoch:  3
Same Different Classification loss: 0.2538214325904846 iteration:  1955 epoch:  3
Male Female Classification loss: 0.12112355977296829 iteration:  1955 epoch:  3
Young Classification loss: 3.8810906410217285 iteration:  1955 epoch:  3
----

-1

from here

to here

In [ ]:
import numpy as np

val_index1 = np.random.permutation(val_index)[:100]
val_dataloader = DataLoader(dataset,batch_size=50,sampler = SubsetRandomSampler(test_index))
val_iter = iter(val_dataloader)
predicted_lables = []
real_labels_g = []
predicted_lables_g = []
real_labels = []
real_labels_y = []
predicted_lables_y = []
predicted_lables_F=[]
predicted_lables_M=[]
real_labels_F = []
real_labels_M = []

total = 0
total_F = 0
total_M = 0
correct1_SD = 0
correct1_MF = 0
correct_M = 0
correct_F = 0
correct1_YO = 0
acc1=[]
acc2=[]
acc_M = []
acc_F = []
acc1_YO = []
for j,dataj in enumerate(val_dataloader):
            input1, input2 , label, gender ,young= data.items()
            input1, input2 = input1[1].to(device), input2[1].to(device)
            labelj = label[1].to(device)
            gender = gender[1].to(device)
            young = young[1].to(device)
            gender = (gender+torch.ones_like(gender).to(device))//2
            young = (young+torch.ones_like(young).to(device))//2

            mu_all = torch.cat((mu_high,mu_low.data),1)
            logvar_all = torch.cat((logvar_high,logvar_low),1)
            class_input = torch.cat((mu_all,logvar_all),1)
            #class_input = mu_all
            class_out_SD = class_net_SD(class_input)
            class_out_MF = class_net_MF(class_input)
            class_out_YO = class_net_YO(class_input)
            _,predicted1_SD = torch.max(class_out_SD.data,1)
            _,predicted1_MF = torch.max(class_out_MF.data,1)
            _,predicted1_YO = torch.max(class_out_YO.data,1)
            # _,predicted2 = torch.max(output_A.data,1)
            total +=labelj.size(0)
            correct1_SD += (predicted1_SD == labelj).sum().item()
            correct1_MF += (predicted1_MF == gender).sum().item()
            # for gender == 0
            correct_F += (predicted1_MF[gender==0] == gender[gender==0]).sum().item()
            total_F += len(gender[gender==0])
            predicted_lables_F.append(torch.Tensor.numpy(predicted1_MF[gender==0].cpu()))
            real_labels_F.append(torch.Tensor.numpy(gender[gender==0].cpu()))

             # for gender == 1
            correct_M += (predicted1_MF[gender==1] == gender[gender==1]).sum().item()
            total_M += gender[gender==1].sum().item()

            predicted_lables_M.append(torch.Tensor.numpy(predicted1_MF[gender==1].cpu()))
            real_labels_M.append(torch.Tensor.numpy(gender[gender==1].cpu()))



            correct1_YO += (predicted1_YO == young).sum().item()
            predicted_lables.append(torch.Tensor.numpy(predicted1_SD.cpu()))
            real_labels.append(torch.Tensor.numpy(labelj.cpu()))

            predicted_lables_g.append(torch.Tensor.numpy(predicted1_MF.cpu()))
            real_labels_g.append(torch.Tensor.numpy(gender.cpu()))
            predicted_lables_y.append(torch.Tensor.numpy(predicted1_YO.cpu()))
            real_labels_y.append(torch.Tensor.numpy(young.cpu()))

            # correct2 += (predicted2 == gender).sum().item()
            print('Accuracy of SD: %d %%'%(100*correct1_SD/total))
            print('Accuracy of MF: %d %%'%(100*correct1_MF/total))
            print('Accuracy of F: %d %%'%(100*correct_F/total_F))
            print('Accuracy of M: %d %%'%(100*correct_M/total_M))

            print('Accuracy of YO: %d %%'%(100*correct1_YO/total))

           
            acc1.append(100*correct1_SD/total)
            acc2.append(100*correct1_MF/total)
            acc1_YO.append(100*correct1_YO/total)
            print('confusion matrix for same differen')
            print(mt.confusion_matrix(np.array(real_labels).flatten(),np.array(predicted_lables).flatten()))
            print('confusion matrix for female')
            print(mt.confusion_matrix(np.array(real_labels_F).flatten(),np.array(predicted_lables_F).flatten()))
            print('confusion matrix for male')
            print(mt.confusion_matrix(np.array(real_labels_M).flatten(),np.array(predicted_lables_M).flatten()))
            print('confusion matrix for male female')
            print(mt.confusion_matrix(np.array(real_labels_g).flatten(),np.array(predicted_lables_g).flatten()))
            print('confusion matrix for young old')
            print(mt.confusion_matrix(np.array(real_labels_y).flatten(),np.array(predicted_lables_y).flatten()))
